<a href="https://colab.research.google.com/github/Leia2000/Bachelorarbeit_Skripte/blob/main/Calculate_IoU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
require(terra)
require(caret)

setwd("C:/BSc_Arbeit/Twigs/Klassifikation/test_zweige")

# Load your binary raster layers
raster1 <- rast("mask1_84_52.tif")
raster2 <- rast("no_probability_9.tif")

# get value range of prediction raster and restrict to 1 and 99% percentile
percentiles <- quantile(values(raster2), probs = c(0.01, 0.99), na.rm = TRUE)
# create 1000 thresholds to test
th <- seq( percentiles[1],percentiles[2],(as.numeric(percentiles[2]-percentiles[1])/1999))

# create empty list to store results
IoU_res <- list()

# iterate through thresholds
for (i in 1:1999){

  # apply current treshold and create binary raster with value 0 or 1
  raster3 <- raster2 > th[i]
  raster3 <- as.numeric(raster3)

  # Calculate the intersection (logical AND)
  intersection <- raster1 & raster3

  # Calculate the union (logical OR)
  union <- raster1 | raster3

  # calculate IoU helpers
  intersection_area <- global(intersection, fun = sum, na.rm = TRUE)[,1]
  union_area <- global(union, fun = sum, na.rm = TRUE)[,1]

  # Calculate IoU
  iou <- intersection_area / union_area

  # calaculate traditional pixel-based confusion matrix
  vec1 <- as.factor(as.vector(raster1))
  ## vec1 hat NaN Werte, die werden zu 0 Werten gewandelt
  vec1 <- as.character(vec1)  # Faktor in Character umwandeln
  nan_count <- sum(vec1 == "NaN")  # "NaN" als String erkennen
  vec1[vec1 == "NaN"] <- "0"  # NaN durch 0 ersetzen
  vec1 <- as.factor(vec1)  # Wieder in Factor umwandeln

  vec2 <- as.factor(as.vector(raster3))
  valid_idx <- !is.na(vec1)  # Nur Werte behalten, die keine NaN sind
  vec1 <- vec1[valid_idx]
  vec2 <- vec2[valid_idx]
  cfm <- confusionMatrix(vec1, vec2)

  # extract kappy value
  cfm$overall[2]

  # Save IoU and kappa value to list
  IoU_res[[i]] <- c(iou, cfm$overall[2])

  # print current iteration (i of 1000 tresholds)
  print(i)


}

# prepare data
res <- do.call(rbind, IoU_res)
# plot kappa values
plot(seq(1,1999,1), res[,2])

# get highest kappa value
best <- which(res[,2] == max(res[,2]))
res[best]
th[best]

# apply threshold of highest kappa value
raster_best <- raster2 > th[best]
# check plots after applying best threshold according to kappa
par(mfrow = c(2,2))

plot(raster_best)
plot(raster1)
plot(raster2)




